# Animal Control Data Profiling
## Rebecca Lewis
### DSC 680 - Project 1

In [1]:
from pandas_profiling import ProfileReport
import pandas as pd
import numpy as np
from geopy.geocoders import GoogleV3
import geopy

In [2]:
clean_df = pd.read_csv('../Data/EDA_dataset.csv')

In [3]:
profile = ProfileReport(clean_df, title="BRACRC Data Profiling Report")

In [4]:
profile.to_widgets()

Summarize dataset:   0%|          | 0/45 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

### Moderate number of missing or null values; need to decide to discard missing observations or exclude columns
- dispatched time
- available time
- arrival time
About the same percentage is missing for each one


- Size? - very low number missing - can recode as Unknown
- Condition - missing can be set as unknown
- temprement - missing can be set as unknown; rename column to temperament


### High number of missing values - eliminate
- request_type - decided to keep this one to use in models that predict the condition and temperament.  using the service code would cause feature leakage.
- Age - Moderate number missing and high number of 0's
- disposition date - not relevant; removing

### Irrelevant features
- file_number
- impound number

### Cardinality Reduction
- Combine all animals other than cats and dogs to other
- Breed has too high of a cardinality to be useful as is.  Will need to take another look after missing values are handled and interactions are evaluated
- Sex - only m, f and u are relevant - need to combine all other values into u

### New Features
- Time between call and dispatch - time between the incident report and dispatched
- Time between call and arrival - time to arrive after incident was reported
- Time between dispatch and arrival - time to get to the scene once dispatched
- time between arrival and available - time to work the incident
- time between call and avaiable - time between the incident was reported and was finished being worked by officer
- time between the initial call and impound time - not all animals were impounded so there may be some sort of future leak here
- collar - create a variable to indicate whether a collar was found on the animal. items will be collar, no collar and unknown.  Missing will go under unknown

### Target Variable
- Disposition - can the outcome be predicted by other variables
- Latitude/longitued - can the region be predicted by other variables.

### Explore missing times

In [5]:
missing_times = clean_df[clean_df['dispatched_time'].isna() |  clean_df['arrival_time'].isna() |  clean_df['available_time'].isna()]

In [6]:
missing_times.head()

,file_number,incident_date,incident_time,dispatched_time,arrival_time,available_time,request_type,impound_number,impound_date,impound_time,...,disposition,disposition_date,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,location_y,lat,long
13669,2012244227,11/29/2012,10:15,NaN,NaN,NaN,NaN,24429,11/29/2012,10:15,...,TRANS CAA,2012-11-29 00:00:00,2012-11-29 10:15:00,NaN,NaN,NaN,2012-11-29 10:15:00,"2567 PROGRESS, BATON ROUGE, LA.",30.537827,-91.164185
13670,2012244227,11/29/2012,10:15,NaN,NaN,NaN,NaN,24428,11/29/2012,10:15,...,TRANS CAA,2012-11-29 00:00:00,2012-11-29 10:15:00,NaN,NaN,NaN,2012-11-29 10:15:00,"2567 PROGRESS, BATON ROUGE, LA.",30.537827,-91.164185
13676,2013255571,06/10/2013,12:23,NaN,NaN,NaN,NaN,36330,06/10/2013,12:23,...,TRANS CAA,2013-06-10 00:00:00,2013-06-10 12:23:00,NaN,NaN,NaN,2013-06-10 12:23:00,"12720 ARLINGFORD, BATON ROUGE, LA.",30.458698,-91.045290
13677,2013255571,06/10/2013,12:23,NaN,NaN,NaN,NaN,36331,06/10/2013,12:24,...,TRANS CAA,2013-06-10 00:00:00,2013-06-10 12:23:00,NaN,NaN,NaN,2013-06-10 12:24:00,"12720 ARLINGFORD, BATON ROUGE, LA.",30.458698,-91.045290
13689,2013250062,03/16/2013,12:28,NaN,NaN,NaN,NaN,33712,03/16/2013,12:28,...,TRANS CAA,2013-03-16 00:00:00,2013-03-16 12:28:00,NaN,NaN,NaN,2013-03-16 12:28:00,"1604 BRIAR RIDGE, BATON ROUGE, LA.",30.358654,-91.071660


In [7]:
missing_times['disposition'].unique()

array(['TRANS CAA', 'EUTHANIZED', 'RETURN TO OWNER', 'RETURN TO WILD',
       'DEAD ON ARRIVAL', 'SPECIAL CASE/OTHER', nan], dtype=object)

In [8]:
missing_times['service_code'].unique()

array(['TRAPPED ANIMAL', 'STRAY', 'INJURED', 'STRAY AN SECURED', 'LOOSE',
       'BITE CASE LOOSE', 'BITE CASE SECURE', 'SPECIAL CASE', 'BITE CASE',
       'ABANDONED ANIMAL', 'UNIT STANDING BY', 'OWNER SIGNED RELEASE',
       'WILD LIVE', 'CRUELTY', 'ANIMAL RESCUE', 'DANGEROUS ANIMAL SIT',
       'SICK ANIMAL', 'LOOSE LIVESTOCK', 'SNAKE', 'SCHOOL CALL',
       'VET PICKUP', 'DOG FIGHTING', 'OWNED'], dtype=object)

In [9]:
missing_times['species'].unique()

array(['DOG', 'CAT', 'WILDLIFE', 'FOWL', 'LIVESTOCK', 'REPTILE', nan,
       'BIRD', 'OTHER'], dtype=object)

In [10]:
clean_df[clean_df['dispatched_time'].isna() &  clean_df['arrival_time'].notna() &  clean_df['available_time'].notna()]

,file_number,incident_date,incident_time,dispatched_time,arrival_time,available_time,request_type,impound_number,impound_date,impound_time,...,disposition,disposition_date,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,location_y,lat,long


There doesn't seem to be a specific value that is associated with the null values. When one of the times is null, the rest appear to be null as well.  I'd like to find out how the time to arrive and complete the incident affect the outcome so I will remove records missing these values.

In [11]:
missing_times.shape, clean_df.shape

#should end up with 43,24 records

((12365, 32), (55889, 32))

In [12]:
clean_df = clean_df.dropna(axis=0, subset=['dispatched_time', 'arrival_time', 'available_time'])
clean_df.shape

(43524, 32)

### Evaluate other missing variables
#### Size

In [13]:
clean_df[clean_df['size'].isna()].head()

,file_number,incident_date,incident_time,dispatched_time,arrival_time,available_time,request_type,impound_number,impound_date,impound_time,...,disposition,disposition_date,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,location_y,lat,long
13712,2012231406,05/16/2012,14:52,14:52,16:30,16:32,NaN,24252,05/16/2012,15:18,...,RETURN TO WILD,2012-05-16 00:00:00,2012-05-16 14:52:00,2012-05-16 14:52:00,2012-05-16 16:30:00,2012-05-16 16:32:00,2012-05-16 15:18:00,"9955 FLORIDA BLVD LOT 16, BATON ROUGE, LA.",30.456661,-91.073340
13763,2014278530,08/06/2014,9:6,9:25,9:39,11:5,NaN,47851,08/06/2014,10:26,...,TRANS CAA,2014-08-06 00:00:00,2014-08-06 09:06:00,2014-08-06 09:25:00,2014-08-06 09:39:00,2014-08-06 11:05:00,2014-08-06 10:26:00,"7134 MELPOMENE, BATON ROUGE, LA. NA",30.584828,-91.120582
13811,2014277827,07/23/2014,8:8,9:15,9:41,9:50,NaN,46998,07/23/2014,9:46,...,TRANS CAA,2014-07-23 00:00:00,2014-07-23 08:08:00,2014-07-23 09:15:00,2014-07-23 09:41:00,2014-07-23 09:50:00,2014-07-23 09:46:00,"5223 BELLE FOUNTAIN, BATON ROUGE, LA. NA",30.388827,-91.196646
13815,2013260064,08/27/2013,8:24,9:14,9:37,10:1,NaN,38467,08/27/2013,9:48,...,TRANS CAA,2013-08-27 00:00:00,2013-08-27 08:24:00,2013-08-27 09:14:00,2013-08-27 09:37:00,2013-08-27 10:01:00,2013-08-27 09:48:00,"6444 JONES CREEK, BATON ROUGE, LA.",30.393470,-91.018220
13851,2013254141,05/20/2013,15:19,15:20,15:42,15:49,NaN,34980,05/20/2013,15:45,...,TRANS CAA,2013-05-20 00:00:00,2013-05-20 15:19:00,2013-05-20 15:20:00,2013-05-20 15:42:00,2013-05-20 15:49:00,2013-05-20 15:45:00,"6667 JUNIPER, BATON ROUGE, LA.",30.511242,-91.124374


In [14]:
clean_df[clean_df['size'].isna()].shape

(995, 32)

In [15]:
#recode as unknown
clean_df['size'] = clean_df['size'].fillna(value='UNKNOWN') 

C:\Users\rvick\Anaconda3\envs\R_Python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [16]:
clean_df['size'].unique()

array(['LARGE', 'SMALL', 'EXTRA LARGE', 'MEDIUM', 'UNKNOWN'], dtype=object)

#### Condition

In [17]:
clean_df[clean_df['condition'].isna()].head()

,file_number,incident_date,incident_time,dispatched_time,arrival_time,available_time,request_type,impound_number,impound_date,impound_time,...,disposition,disposition_date,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,location_y,lat,long
13721,2013264864,11/12/2013,9:31,10:36,10:57,11:7,NaN,41801,11/12/2013,11:6,...,TRANS CAA,2013-11-12 00:00:00,2013-11-12 09:31:00,2013-11-12 10:36:00,2013-11-12 10:57:00,2013-11-12 11:07:00,2013-11-12 11:06:00,"13934 AIRLINE, BATON ROUGE, LA. NA",30.367536,-91.028621
13732,2013254719,05/29/2013,14:10,14:20,14:34,15:11,NaN,35666,05/29/2013,14:56,...,TRANS CAA,2013-05-29 00:00:00,2013-05-29 14:10:00,2013-05-29 14:20:00,2013-05-29 14:34:00,2013-05-29 15:11:00,2013-05-29 14:56:00,"3405 LINDEN, BATON ROUGE, LA.",30.486510,-91.155630
13753,2012232258,06/01/2012,14:25,14:26,14:43,14:51,NaN,24094,06/01/2012,14:49,...,TRANS CAA,2012-06-01 00:00:00,2012-06-01 14:25:00,2012-06-01 14:26:00,2012-06-01 14:43:00,2012-06-01 14:51:00,2012-06-01 14:49:00,"111 GATEBRIAR, BATON ROUGE, LA.",30.577703,-91.206630
13755,2012229320,04/10/2012,9:36,9:40,9:59,10:27,NaN,22390,04/10/2012,10:19,...,TRANS CAA,2012-04-10 00:00:00,2012-04-10 09:36:00,2012-04-10 09:40:00,2012-04-10 09:59:00,2012-04-10 10:27:00,2012-04-10 10:19:00,"976 MAYFLOWER, BATON ROUGE, LA.",30.440971,-91.179634
13780,2013265281,11/20/2013,9:10,9:12,9:44,10:30,NaN,41523,11/20/2013,9:50,...,TRANS CAA,2013-11-20 00:00:00,2013-11-20 09:10:00,2013-11-20 09:12:00,2013-11-20 09:44:00,2013-11-20 10:30:00,2013-11-20 09:50:00,"13821 GREEN FRANKLIN, BATON ROUGE, LA. NA",30.451468,-91.187147


In [18]:
clean_df[clean_df['condition'].isna()].shape

(1724, 32)

In [19]:
#recode as unknown
clean_df['condition'] = clean_df['condition'].fillna(value='UNKNOWN') 

C:\Users\rvick\Anaconda3\envs\R_Python\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
clean_df['condition'].unique()

array(['FAIR', 'EMACIATED', 'GOOD', 'UNKNOWN', 'POOR', 'EXCELLENT'],
      dtype=object)

#### Temperament

In [21]:
clean_df[clean_df['temperment'].isna()].head()

,file_number,incident_date,incident_time,dispatched_time,arrival_time,available_time,request_type,impound_number,impound_date,impound_time,...,disposition,disposition_date,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,location_y,lat,long
13707,2012230285,04/26/2012,10:4,10:16,10:37,10:41,NaN,24125,04/26/2012,10:40,...,RETURN TO OWNER,2012-04-26 00:00:00,2012-04-26 10:04:00,2012-04-26 10:16:00,2012-04-26 10:37:00,2012-04-26 10:41:00,2012-04-26 10:40:00,"8022 ANTIOCH, BATON ROUGE, LA.",30.379564,-91.007385
13753,2012232258,06/01/2012,14:25,14:26,14:43,14:51,NaN,24094,06/01/2012,14:49,...,TRANS CAA,2012-06-01 00:00:00,2012-06-01 14:25:00,2012-06-01 14:26:00,2012-06-01 14:43:00,2012-06-01 14:51:00,2012-06-01 14:49:00,"111 GATEBRIAR, BATON ROUGE, LA.",30.577703,-91.206630
13755,2012229320,04/10/2012,9:36,9:40,9:59,10:27,NaN,22390,04/10/2012,10:19,...,TRANS CAA,2012-04-10 00:00:00,2012-04-10 09:36:00,2012-04-10 09:40:00,2012-04-10 09:59:00,2012-04-10 10:27:00,2012-04-10 10:19:00,"976 MAYFLOWER, BATON ROUGE, LA.",30.440971,-91.179634
13763,2014278530,08/06/2014,9:6,9:25,9:39,11:5,NaN,47851,08/06/2014,10:26,...,TRANS CAA,2014-08-06 00:00:00,2014-08-06 09:06:00,2014-08-06 09:25:00,2014-08-06 09:39:00,2014-08-06 11:05:00,2014-08-06 10:26:00,"7134 MELPOMENE, BATON ROUGE, LA. NA",30.584828,-91.120582
13815,2013260064,08/27/2013,8:24,9:14,9:37,10:1,NaN,38467,08/27/2013,9:48,...,TRANS CAA,2013-08-27 00:00:00,2013-08-27 08:24:00,2013-08-27 09:14:00,2013-08-27 09:37:00,2013-08-27 10:01:00,2013-08-27 09:48:00,"6444 JONES CREEK, BATON ROUGE, LA.",30.393470,-91.018220


In [22]:
#fix temperament spelling
clean_df = clean_df.rename(columns={'temperment':'temperament'})

In [23]:
clean_df.columns

Index(['file_number', 'incident_date', 'incident_time', 'dispatched_time',
       'arrival_time', 'available_time', 'request_type', 'impound_number',
       'impound_date', 'impound_time', 'location_x', 'municipality_',
       'zip_code', 'service_code', 'species', 'breed', 'sex', 'size', 'age',
       'condition', 'temperament', 'collar', 'disposition', 'disposition_date',
       'incident_datetime', 'dispatched_datetime', 'arrival_datetime',
       'available_datetime', 'impound_datetime', 'location_y', 'lat', 'long'],
      dtype='object')

In [24]:
#recode as unknown
clean_df['temperament'] = clean_df['temperament'].fillna(value='UNKNOWN') 

In [25]:
#recode as unknown
clean_df['temperament'].unique()

array(['FRIENDLY', 'NORMAL', 'UNKNOWN', 'NERVOUS', 'DANGEROUS', 'SCARED'],
      dtype=object)

#### Request Type

In [26]:
clean_df['request_type']= clean_df['request_type'].fillna(value='UNKNOWN')

In [27]:
clean_df.request_type.unique()

array(['CRUELTY CASE', 'TRAPPED ANIMAL', 'STRAY ANIMAL SECURED',
       'OWNER SIGNED RELEASE', 'INJURED ANIMAL', 'UNKNOWN',
       'UNIT STANDING BY', 'BITE CASE', 'PICK UP TRAP', 'LOOSE ANIMAL',
       'LOOSE LIVE STOCK', 'SICK ANIMAL', 'DANGEROUS ANIMAL SITUATION',
       'VET PICKUP', 'ANIMAL RESCUE', 'WILD LIFE', 'SPECIAL CASE',
       'DELIVER TRAP', 'SNAKE', 'OTHER', 'PATROL AREA', 'CRUELTY RECHECK',
       'VET TRANSFER', 'DELIVERED TAGS', 'WARNING NOTICE',
       'IN HOME SIGNED', 'TRAP', 'ISSUE SUMMONS', 'RECHECK',
       'BARKING DOG', 'DOG FIGHTING', 'STRAY ANIMAL', 'PITBULL',
       'SCHOOL CALL', 'ABANDONED ANIMAL', 'STRAY',
       'PICK UP / DELIVER PACKAGE'], dtype=object)

In [28]:
clean_df['request_type'] = clean_df['request_type'].replace(['LOOSE LIVE STOCK', 'WILD LIFE', 'SNAKE'], 'NON-DOMESTIC ANIMALS')
clean_df['request_type'] = clean_df['request_type'].replace(['VET TRANSFER', 'VET PICKUP'], 'VET TRANSFER/PICKUP')
clean_df['request_type'] = clean_df['request_type'].replace(['STRAY ANIMAL SECURED', 'STRAY ANIMAL'], 'STRAY')
clean_df['request_type'] = clean_df['request_type'].replace(['PICK UP TRAP', 'DELIVER TRAP'], 'PICKUP/DELIVER TRAP')


### Drop Columns

In [29]:
clean_df = clean_df.drop(columns=['age', 'disposition_date', 'file_number', 'impound_number'])

In [30]:
clean_df.shape

(43524, 28)

### Cardinality
#### Species

In [31]:
clean_df.species.value_counts()

DOG          21357
CAT          11175
WILDLIFE      9701
LIVESTOCK      344
REPTILE        314
FOWL           309
BIRDS          132
BIRD           102
UNKNOWN         33
OTHER           31
EXOTIC           6
Name: species, dtype: int64

I would like to keep other as a category and lump all animals besides cats and dogs in other.  Unknown does not make sense as a specials unless the animal is badly decomposed.  I am going to exclude them from this analysis.

In [32]:
clean_df = clean_df[clean_df['species'] != 'UNKNOWN']

In [33]:
clean_df['species'] = clean_df['species'].replace(['WILDLIFE', 'LIVESTOCK', 'REPTILE', 'FOWL','BIRDS', 'BIRD', 'EXOTIC'], 'OTHER')

In [34]:
#drop observations where species is null
clean_df = clean_df.dropna(axis=0, subset=['species'])

In [35]:
clean_df.species.value_counts()

DOG      21357
CAT      11175
OTHER    10939
Name: species, dtype: int64

#### Sex

In [36]:
clean_df.sex.value_counts()

U    12846
M    11391
F    10305
K        4
1        2
N        1
0        1
S        1
D        1
Name: sex, dtype: int64

In [37]:
clean_df['sex'] = clean_df['sex'].replace(['K', '1', '0', 'D','N', 'S'], 'U')

In [38]:
clean_df.sex.value_counts()

U    12856
M    11391
F    10305
Name: sex, dtype: int64

#### Breed for dogs

In [39]:
clean_df[clean_df['species'] == 'DOG']['breed'].unique()

array(['PITBULL', 'YORKSHIRE TERRIER', 'DOBERMAN PINSCHER',
       'GERMAN SHEPHERD', 'XCHOW', 'TERRIER', 'XCHIHUAHUA',
       'MINIATURE PINSCHER', 'XPIT BULL', 'XHOUND', 'XLABRADOR',
       'SHIH TZU', 'HOUND', 'ROTTWEILLER', 'BOXER', 'XTERRIER',
       'LABRADOR RETRIEVER', 'XSHEPHERD', 'UNKNOWN', 'XDACHSHUND',
       'XDOBERMAN', 'XGERMAN SHEPHERD', 'XCATAHOULA', 'POODLE MIN/TOY',
       'PIT', 'AUSTRALIAN SHEPHERD', 'CHIHUAHUA', 'POINTER', 'XHUSKY',
       'XBEAGLE', 'SHEPHERD', 'XDALMATION', 'MIXED', 'XROTTWEILER', 'DSH',
       'XHEALER', 'XBORDER COLLIE', 'CATAHOULA HOUND', 'GOLDEN RETRIEVER',
       'RABBIT', 'XSAINT BERNARD', 'XAMERICAN BULLDOG', 'POMERANIAN',
       'XPOODLE', 'HUSKY', 'XCOLLIE', 'DACHSHUND', 'JACK RUSSEL TERRIER',
       'BLUETICK HOUND', 'BEAGLE', 'XPUG', 'XAUSTRALIAN SHEPHERD',
       'AMERICAN BULLDOG', 'PEKINGESE', 'SCHNAUZER', 'PUG', 'POSSUM',
       'AKITA', 'XBOXER', 'BULLDOG ENGLISH', 'RETRIEVER', 'XRETRIEVER',
       'MALTESE', 'CHOW CHOW', 'GREAT 

In [40]:
# clean up X in front of some breeds
clean_df['breed'] = clean_df['breed'].str.lstrip('X')

In [41]:
clean_df[clean_df['species'] == 'DOG']['breed'].unique()

array(['PITBULL', 'YORKSHIRE TERRIER', 'DOBERMAN PINSCHER',
       'GERMAN SHEPHERD', 'CHOW', 'TERRIER', 'CHIHUAHUA',
       'MINIATURE PINSCHER', 'PIT BULL', 'HOUND', 'LABRADOR', 'SHIH TZU',
       'ROTTWEILLER', 'BOXER', 'LABRADOR RETRIEVER', 'SHEPHERD',
       'UNKNOWN', 'DACHSHUND', 'DOBERMAN', 'CATAHOULA', 'POODLE MIN/TOY',
       'PIT', 'AUSTRALIAN SHEPHERD', 'POINTER', 'HUSKY', 'BEAGLE',
       'DALMATION', 'MIXED', 'ROTTWEILER', 'DSH', 'HEALER',
       'BORDER COLLIE', 'CATAHOULA HOUND', 'GOLDEN RETRIEVER', 'RABBIT',
       'SAINT BERNARD', 'AMERICAN BULLDOG', 'POMERANIAN', 'POODLE',
       'COLLIE', 'JACK RUSSEL TERRIER', 'BLUETICK HOUND', 'PUG',
       'PEKINGESE', 'SCHNAUZER', 'POSSUM', 'AKITA', 'BULLDOG ENGLISH',
       'RETRIEVER', 'MALTESE', 'CHOW CHOW', 'GREAT DANE', 'OTHER',
       'SCOTTISH TERRIER', 'WHIPPET', 'BASSET', 'RAT TERRIER', 'MASTIFF',
       'BOSTON', ' RETRIEVER', 'DMH', 'LHASA APSO', 'MINK', 'CURR',
       'SPRINGER SPANIEL', 'CAIRN TERRIER', 'BULL TERRIE

In [42]:
clean_df['breed'] = clean_df['breed'].replace(['PIT', 'PITBULL', 'pit'], 'PIT BULL')
clean_df['breed'] = clean_df['breed'].replace('DOBERMAN', 'DOBERMAN PINSCHER')
clean_df['breed'] = clean_df['breed'].replace('LABRADOR', 'LABRADOR RETRIEVER')
clean_df['breed'] = clean_df['breed'].replace(' RETRIEVER', 'RETRIEVER')
clean_df['breed'] = clean_df['breed'].replace('BOSTON', 'BOSTON TERRIER')
clean_df['breed'] = clean_df['breed'].replace('ROTTWEILLER', 'ROTTWEILER')
clean_df['breed'] = clean_df['breed'].replace('CATAHOULA HOUND', 'CATAHOULA')
clean_df['breed'] = clean_df['breed'].replace(['POODLE MIN/TOY','POODLE STANDARD', 'POODLE (MIN/TOY)', 'POODLE -STANDARD'], 'POODLE')
clean_df['breed'] = clean_df['breed'].replace('CHOW', 'CHOW CHOW')
clean_df['breed'] = clean_df['breed'].replace('BASSETT', 'BASSETT HOUND')
clean_df['breed'] = clean_df['breed'].replace('HUSKY', 'SIBERIAN HUSKY')
clean_df['breed'] = clean_df['breed'].replace('SHARPEI', 'SHAR PEI')
clean_df['breed'] = clean_df['breed'].replace('HEALER', 'AUSTRALIAN BLUE HEALER')
clean_df['breed'] = clean_df['breed'].replace(' GERMAN SHEPHERD', 'GERMAN SHEPHERD')

In [43]:
clean_df[clean_df['species'] == 'DOG']['breed'].unique()

array(['PIT BULL', 'YORKSHIRE TERRIER', 'DOBERMAN PINSCHER',
       'GERMAN SHEPHERD', 'CHOW CHOW', 'TERRIER', 'CHIHUAHUA',
       'MINIATURE PINSCHER', 'HOUND', 'LABRADOR RETRIEVER', 'SHIH TZU',
       'ROTTWEILER', 'BOXER', 'SHEPHERD', 'UNKNOWN', 'DACHSHUND',
       'CATAHOULA', 'POODLE', 'AUSTRALIAN SHEPHERD', 'POINTER',
       'SIBERIAN HUSKY', 'BEAGLE', 'DALMATION', 'MIXED', 'DSH',
       'AUSTRALIAN BLUE HEALER', 'BORDER COLLIE', 'GOLDEN RETRIEVER',
       'RABBIT', 'SAINT BERNARD', 'AMERICAN BULLDOG', 'POMERANIAN',
       'COLLIE', 'JACK RUSSEL TERRIER', 'BLUETICK HOUND', 'PUG',
       'PEKINGESE', 'SCHNAUZER', 'POSSUM', 'AKITA', 'BULLDOG ENGLISH',
       'RETRIEVER', 'MALTESE', 'GREAT DANE', 'OTHER', 'SCOTTISH TERRIER',
       'WHIPPET', 'BASSET', 'RAT TERRIER', 'MASTIFF', 'BOSTON TERRIER',
       'DMH', 'LHASA APSO', 'MINK', 'CURR', 'SPRINGER SPANIEL',
       'CAIRN TERRIER', 'BULL TERRIER', 'CHICKEN', 'RHODESIAN RIDGEBACK',
       'COCKER SPANIEL', 'SPANIEL', 'ENGLISH BULLDOG

In [44]:
clean_df['breed'] = clean_df['breed'].replace(['BULLDOG (ENGLISH)', 'BULLDOG ENGLISH'], 'ENGLISH BULLDOG')
clean_df['breed'] = clean_df['breed'].replace('MASTIFF', 'BULL MASTIFF')

In [45]:
clean_df[clean_df['species'] == 'DOG']['breed'].unique()

array(['PIT BULL', 'YORKSHIRE TERRIER', 'DOBERMAN PINSCHER',
       'GERMAN SHEPHERD', 'CHOW CHOW', 'TERRIER', 'CHIHUAHUA',
       'MINIATURE PINSCHER', 'HOUND', 'LABRADOR RETRIEVER', 'SHIH TZU',
       'ROTTWEILER', 'BOXER', 'SHEPHERD', 'UNKNOWN', 'DACHSHUND',
       'CATAHOULA', 'POODLE', 'AUSTRALIAN SHEPHERD', 'POINTER',
       'SIBERIAN HUSKY', 'BEAGLE', 'DALMATION', 'MIXED', 'DSH',
       'AUSTRALIAN BLUE HEALER', 'BORDER COLLIE', 'GOLDEN RETRIEVER',
       'RABBIT', 'SAINT BERNARD', 'AMERICAN BULLDOG', 'POMERANIAN',
       'COLLIE', 'JACK RUSSEL TERRIER', 'BLUETICK HOUND', 'PUG',
       'PEKINGESE', 'SCHNAUZER', 'POSSUM', 'AKITA', 'ENGLISH BULLDOG',
       'RETRIEVER', 'MALTESE', 'GREAT DANE', 'OTHER', 'SCOTTISH TERRIER',
       'WHIPPET', 'BASSET', 'RAT TERRIER', 'BULL MASTIFF',
       'BOSTON TERRIER', 'DMH', 'LHASA APSO', 'MINK', 'CURR',
       'SPRINGER SPANIEL', 'CAIRN TERRIER', 'BULL TERRIER', 'CHICKEN',
       'RHODESIAN RIDGEBACK', 'COCKER SPANIEL', 'SPANIEL', 'WEIMERANER

In [46]:
clean_df[clean_df['species'] == 'DOG']['breed'].value_counts()

PIT BULL                 8500
LABRADOR RETRIEVER       2971
TERRIER                  1850
GERMAN SHEPHERD          1272
SHEPHERD                  895
                         ... 
MINK                        1
PEACOCK                     1
WELSH CORGI                 1
FOXHOUND                    1
BOURVIER DES FLANDERS       1
Name: breed, Length: 90, dtype: int64

In [47]:
clean_df[clean_df['species'] == 'CAT']['breed'].unique()

array(['DMH', 'DSH', 'SHAR PEI', 'SIAMESE', 'DLH', 'CHIHUAHUA', 'UNKNOWN',
       'PIT BULL', 'POSSUM', 'CHOW CHOW', 'GERMAN SHEPHERD', 'SHEPHERD',
       'CAIRN TERRIER', 'LABRADOR RETRIEVER', 'DONKEY', 'SIBERIAN HUSKY',
       'OTHER', 'MIXED SIAMESE', nan, 'HIMALAYAN', 'PERSIAN', 'MANX',
       'DUCK DOMESTIC', 'GREAT DANE', 'TERRIER', 'HAMSTER'], dtype=object)

There are some breeds that do not match with the species.  These can affect any models.  I'm going to remove rows with non dog breeds from dog observations and non cat breeds from cat observations. Same with other animals.

In [48]:
clean_df[clean_df['species'] == 'OTHER']['breed'].unique()

array(['POSSUM', 'SNAKE DOMESTIC NON POISONOUS', 'OTHER', 'RACCOON',
       'SQUIRREL', 'HORSE', 'BAT', 'HAWK', 'TURTLE', 'UNKNOWN',
       'ARMADILLO', 'PIT BULL', 'BEAVER', 'SKUNK', 'CHICKEN', 'RABBIT',
       'ROOSTER', 'DMH', 'POINTER', 'PIG', 'GOAT', 'OWL', 'RETRIEVER',
       'NUTRIA', 'TERRIER', 'GOOSE', 'DUCK DOMESTIC', 'COW',
       'SNAKE DOMESTIC POISONOUS', 'DSH', 'POODLE', 'OSTRICH', 'PARROT',
       'COYOTE', 'CROW', 'FERRET', 'GUINEA', 'CHIHUAHUA',
       'LABRADOR RETRIEVER', 'GUINEA PIG', nan, 'DONKEY', 'RAT', 'MOUSE',
       'WOLF', 'WALKER HOUND', 'LIZARD', 'DACHSHUND', 'SHEEP',
       'GERMAN SHEPHERD', 'ENGLISH BULLDOG', 'MULE', 'FISH', 'HAMSTER',
       'IGUANA'], dtype=object)

In [49]:
#FROM OTHER REMOVE
# PIT BULL, DMH, POINTER, RETRIEVER, TERRIER, DSH, POODLE, CHIHUAHUA, LABORADOR RETRIEVER, WALKER HOUD, DACHSHUND
# GERMAN SHEPHERD, ENGLISH BULLDOG

# FROM CAT REMOVE:
# SHAR PEI, CHIHUAHUA, PIT BULL, POSSUM, CHOW CHOW, GERMAN SHEPHERD, SHEPHERD, CAIRN TERRIER, LABORADOR RETRIEVER, 
# 'DONKEY', 'SIBERIAN HUSKY', 'DUCK DOMESTIC', 'GREAT DANE', 'TERRIER'
# CHANGE NULL TO UNKNOWN

#FROM DOG REMOVE:
#DSH, RABBIT, POSSUM, MINK, DMH, CHICKEN, RACCOON, PEACOCK, PARROT

In [50]:
#drop other observations where breed doesn't make sens

drop_indexes = clean_df.index[(clean_df['species'] == 'OTHER') & (clean_df['breed'].isin(['PIT BULL', 'DMH', 'POINTER', 'RETRIEVER', 'TERRIER', 'DSH', 'POODLE', 
                                                     'CHIHUAHUA', 'LABORADOR RETRIEVER', 'WALKER HOUND', 'DACHSHUND',
                                                    'GERMAN SHEPHERD', 'ENGLISH BULLDOG']))].tolist()

clean_df = clean_df.drop(drop_indexes)
clean_df.shape


(43449, 28)

In [51]:
drop_indexes = clean_df.index[(clean_df['species'] == 'CAT') & (clean_df['breed'].isin(['SHAR PEI', 'CHIHUAHUA', 'PIT BULL', 
                                                                                        'POSSUM', 'CHOW CHOW', 'GERMAN SHEPHERD',
                                                                                        'SHEPHERD', 'CAIRN TERRIER', 'LABORADOR RETRIEVER',
                                                                                        'DONKEY', 'SIBERIAN HUSKY', 'DUCK DOMESTIC', 
                                                                                        'GREAT DANE', 'TERRIER']))].tolist()

clean_df = clean_df.drop(drop_indexes)
clean_df.shape


(43421, 28)

In [52]:
drop_indexes = clean_df.index[(clean_df['species'] == 'DOG') & (clean_df['breed'].isin(['DSH', 'RABBIT', 'POSSUM', 'MINK', 'DMH', 'CHICKEN', 
                                                                         'RACCOON', 'PEACOCK', 'PARROT']))].tolist()

clean_df = clean_df.drop(drop_indexes)
clean_df.shape

(43388, 28)

In [53]:
clean_df['breed'] = clean_df['breed'].fillna(value='UNKNOWN')

In [54]:
clean_df['breed'].unique()

array(['PIT BULL', 'POSSUM', 'YORKSHIRE TERRIER', 'DOBERMAN PINSCHER',
       'GERMAN SHEPHERD', 'DMH', 'DSH', 'SNAKE DOMESTIC NON POISONOUS',
       'OTHER', 'CHOW CHOW', 'TERRIER', 'CHIHUAHUA', 'MINIATURE PINSCHER',
       'HOUND', 'LABRADOR RETRIEVER', 'SHIH TZU', 'SIAMESE', 'RACCOON',
       'SQUIRREL', 'ROTTWEILER', 'HORSE', 'BOXER', 'DLH', 'SHEPHERD',
       'UNKNOWN', 'DACHSHUND', 'BAT', 'HAWK', 'CATAHOULA', 'POODLE',
       'AUSTRALIAN SHEPHERD', 'TURTLE', 'POINTER', 'SIBERIAN HUSKY',
       'BEAGLE', 'ARMADILLO', 'BEAVER', 'SKUNK', 'CHICKEN', 'DALMATION',
       'MIXED', 'AUSTRALIAN BLUE HEALER', 'BORDER COLLIE', 'RABBIT',
       'GOLDEN RETRIEVER', 'ROOSTER', 'SAINT BERNARD', 'AMERICAN BULLDOG',
       'POMERANIAN', 'COLLIE', 'PIG', 'JACK RUSSEL TERRIER',
       'BLUETICK HOUND', 'PUG', 'PEKINGESE', 'SCHNAUZER', 'AKITA',
       'ENGLISH BULLDOG', 'RETRIEVER', 'MALTESE', 'GOAT', 'GREAT DANE',
       'OWL', 'SCOTTISH TERRIER', 'WHIPPET', 'BASSET', 'NUTRIA', 'GOOSE',
       'DUC

In [55]:
clean_df.drop_duplicates()

,incident_date,incident_time,dispatched_time,arrival_time,available_time,request_type,impound_date,impound_time,location_x,municipality_,...,collar,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,location_y,lat,long
0,12/27/2016,15:30,16:06,16:17,16:38,CRUELTY CASE,12/27/2016,16:20,3366 SYCAMORE,BATON ROUGE,...,NYLON,TRANS CAA,2016-12-27 15:30:00,2016-12-27 16:06:00,2016-12-27 16:17:00,2016-12-27 16:38:00,2016-12-27 16:20:00,"3366 SYCAMORE, BATON ROUGE, LA.",30.494640,-91.156160
1,12/28/2016,10:06,10:07,10:15,10:23,TRAPPED ANIMAL,12/28/2016,10:21,2582 WOODLAND RIDGE,BATON ROUGE,...,UNKNOWN,RETURN TO WILD,2016-12-28 10:06:00,2016-12-28 10:07:00,2016-12-28 10:15:00,2016-12-28 10:23:00,2016-12-28 10:21:00,"2582 WOODLAND RIDGE, BATON ROUGE, LA.",30.431417,-91.141070
2,12/28/2016,09:03,09:03,09:27,09:45,STRAY,12/28/2016,09:41,4463 MAIN,ZACHARY,...,LEATHER,TRANS CAA,2016-12-28 09:03:00,2016-12-28 09:03:00,2016-12-28 09:27:00,2016-12-28 09:45:00,2016-12-28 09:41:00,"4463 MAIN, ZACHARY, LA.",30.648540,-91.156770
3,12/28/2016,15:05,15:45,16:07,16:25,STRAY,12/28/2016,16:20,299 VAN BUREN,BATON ROUGE,...,UNKNOWN,TRANS CAA,2016-12-28 15:05:00,2016-12-28 15:45:00,2016-12-28 16:07:00,2016-12-28 16:25:00,2016-12-28 16:20:00,"299 VAN BUREN, BATON ROUGE, LA.",30.429104,-91.185036
4,12/28/2016,10:03,10:10,10:35,10:52,OWNER SIGNED RELEASE,12/28/2016,10:41,6621 MARIONETTE,BATON ROUGE,...,NYLON,TRANS CAA,2016-12-28 10:03:00,2016-12-28 10:10:00,2016-12-28 10:35:00,2016-12-28 10:52:00,2016-12-28 10:41:00,"6621 MARIONETTE, BATON ROUGE, LA.",30.529593,-91.125534
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55884,09/13/2019,10:07,10:09,10:24,10:41,ABANDONED ANIMAL,09/13/2019,10:34,741 29TH,BATON ROUGE,...,NYLON,TRANS CAA,2019-09-13 10:07:00,2019-09-13 10:09:00,2019-09-13 10:24:00,2019-09-13 10:41:00,2019-09-13 10:34:00,"741 29TH, BATON ROUGE, LA. 70802",30.455312,-91.159822
55885,06/04/2019,10:24,10:52,10:55,11:14,STRAY,06/04/2019,11:05,14513 BRETON,BATON ROUGE,...,NYLON,TRANS CAA,2019-06-04 10:24:00,2019-06-04 10:52:00,2019-06-04 10:55:00,2019-06-04 11:14:00,2019-06-04 11:05:00,"14513 BRETON, BATON ROUGE, LA.",30.448734,-91.024124
55886,10/20/2016,08:09,09:02,09:16,09:26,STRAY,10/20/2016,09:20,9821 FLORIDA BLVD,BATON ROUGE,...,NYLON,TRANS CAA,2016-10-20 08:09:00,2016-10-20 09:02:00,2016-10-20 09:16:00,2016-10-20 09:26:00,2016-10-20 09:20:00,"9821 FLORIDA BLVD, BATON ROUGE, LA.",30.456173,-91.076430
55887,03/21/2018,16:14,16:14,16:35,16:45,STRAY,03/21/2018,16:41,9335 REDWOOD LAKE,ZACHARY,...,NYLON,TRANS CAA,2018-03-21 16:14:00,2018-03-21 16:14:00,2018-03-21 16:35:00,2018-03-21 16:45:00,2018-03-21 16:41:00,"9335 REDWOOD LAKE, ZACHARY, LA.",30.657309,-91.107570


### New Features

In [56]:
# Time between call and dispatch - time between the incident report and dispatched
# Time between call and arrival - time to arrive after incident was reported
# Time between dispatch and arrival - time to get to the scene once dispatched
# time between arrival and available - time to work the incident
# time between call and avaiable - time between the incident was reported and was finished being worked by officer
# time between the initial call and impound time - not all animals were impounded so there may be some sort of future leak here
# collar - create a variable to indicate whether a collar was found on the animal. items will be collar, no collar and unknown. Missing will go under unknown

clean_df[['dispatched_datetime','incident_datetime','arrival_datetime', 'available_datetime','impound_datetime']] = clean_df[['dispatched_datetime','incident_datetime','arrival_datetime', 'available_datetime','impound_datetime']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
call_to_dispatch = clean_df['dispatched_datetime'] - clean_df['incident_datetime']
call_to_arrival = clean_df['arrival_datetime'] - clean_df['incident_datetime']
dispatch_to_arrival = clean_df['arrival_datetime'] - clean_df['dispatched_datetime']
arrival_to_complete = clean_df['available_datetime'] - clean_df['arrival_datetime']
call_to_complete = clean_df['available_datetime'] - clean_df['incident_datetime']
call_to_impound = clean_df['impound_datetime'] - clean_df['incident_datetime']


In [57]:
clean_df['call_to_dispatch_sec'] = call_to_dispatch.dt.total_seconds()
clean_df['call_to_arrival_sec'] = call_to_arrival.dt.total_seconds()
clean_df['dispatch_to_arrival_sec'] = dispatch_to_arrival.dt.total_seconds()
clean_df['arrival_to_complete_sec'] = arrival_to_complete.dt.total_seconds()
clean_df['call_to_complete_sec'] = call_to_complete.dt.total_seconds()
clean_df['call_to_impound_sec'] = call_to_impound.dt.total_seconds()


#### Collar

In [58]:
#determine categories that indicate the animal was found with a collar
clean_df.collar.unique()

array(['NYLON', 'UNKNOWN', 'LEATHER', 'NONE', 'SPIKE', nan, 'ROPE',
       'HALTERNYLO', 'LEASH', 'PLASTIC', 'CHOKE', 'CHAIN', 'FLEA',
       'CLOTH', 'ROPE(PK/YW', 'NON', 'NYLONN', 'SILVER', 'STRIPES/NY',
       'NYLON/FLEA', 'LEATHER/ST', 'COLLAR', 'HARNESS', 'NYLON-BONE',
       'NYLON PINK', 'HALTER', 'WIRE', 'METAL', 'PINCH', 'SHIRT',
       'CHOK CHAIN', 'LTHR STUD', 'N', 'NYLON/STUB', 'NLYON', 'STRING',
       'NYLON SPIK', 'LEATHER SP', 'BANDANA', 'NEON', 'SHOCK', 'RUBBER',
       'BLK', 'NYLON STUD', 'BELT', 'VINYL', 'CHOKER', 'CHAIN/CHRO',
       'LSU', 'NYLON/', 'LARGE CHAI', 'LEATHER FL', 'BLUE PINK',
       'NYLON/METL', 'NTYLON', 'NYLONCHAIN', 'NYLON/BUCK', 'CHOKE CHAI',
       'BLUE CAMO', 'NYLON FOUN', 'NYLON/DAMA', 'PLEATHER', 'BRIDLE',
       'BARKING', 'SWEATER', 'SPIKED', 'CAMO NYLON', 'CHAIN MET',
       'NYLON/GRAY', 'NYLON/BELL', 'CHOKECHAIN', 'NYLON/FLEE', 'SCARF',
       'CHAINWHTBL', 'NYLON/CAMO', 'CAMO', 'PINCH COLL', 'STUDDED',
       'NYLON HARN', 'RUBBERIS

In [59]:
#found with collar
collars = ['NYLON', 'LEATHER', 'SPIKE', 'ROPE', 'HALTERNYLO', 'LEASH', 'PLASTIC', 'FLEA', 'CLOTH', 'NYLONN', 'SILVER', 
           'STRIPES/NY','NYLON/FLEA', 'LEATHER/ST', 'COLLAR', 'HARNESS', 'NYLON-BONE','NYLON PINK', 'HALTER', 'METAL', 
           'LTHR STUD', 'NYLON/STUB', 'NLYON', 'NYLON SPIK', 'LEATHER SP', 'BANDANA', 'NEON', 'RUBBER', 'BLK', 'NYLON STUD',
           'VINYL', 'CHAIN/CHRO', 'LSU', 'NYLON/',  'LEATHER FL', 'BLUE PINK','NYLON/METL', 'NTYLON', 'NYLONCHAIN', 
           'NYLON/BUCK',  'BLUE CAMO', 'NYLON FOUN', 'NYLON/DAMA', 'PLEATHER', 'BRIDLE', 'BARKING', 'SPIKED', 'CAMO NYLON', 
           'NYLON/GRAY', 'NYLON/BELL', 'NYLON/FLEE', 'SCARF', 'NYLON/CAMO', 'CAMO', 'STUDDED','NYLON HARN', 'RUBBERISH', 
           'HARDNESS', 'NYLOM', 'NYLON/PLAS', 'NYLON HALT', 'NYLON CAMO', 'NYLON/PATT', 'NYLON/CHAI', 'SHIRT', 'SWEATER']

#found with questionable collar
tethers = ['CHOKE', 'CHAIN', 'ROPE(PK/YW', 'WIRE',  'PINCH', 'CHOK CHAIN','STRING', 'SHOCK',  'BELT',  'CHOKER', 'LARGE CHAI', 
           'CHOKE CHAI', 'CHAIN MET', 'CHOKECHAIN', 'CHAINWHTBL', 'PINCH COLL', 'NYLON SHOC', 'CORD',  'SHOCK TYPE', 
           'N SHOE STR','CABLE', 'CHOKE CH']
           
clean_df['with_collar'] = np.where(clean_df['collar'].isin(collars), True, False)
clean_df['with_tether'] = np.where(clean_df['collar'].isin(tethers), True, False)

In [60]:
clean_df[['collar', 'with_collar', 'with_tether']].head()

,collar,with_collar,with_tether
0,NYLON,True,False
1,UNKNOWN,False,False
2,LEATHER,True,False
3,UNKNOWN,False,False
4,NYLON,True,False


### Final Cleanup Before Visualizations

In [61]:
clean_df.columns

Index(['incident_date', 'incident_time', 'dispatched_time', 'arrival_time',
       'available_time', 'request_type', 'impound_date', 'impound_time',
       'location_x', 'municipality_', 'zip_code', 'service_code', 'species',
       'breed', 'sex', 'size', 'condition', 'temperament', 'collar',
       'disposition', 'incident_datetime', 'dispatched_datetime',
       'arrival_datetime', 'available_datetime', 'impound_datetime',
       'location_y', 'lat', 'long', 'call_to_dispatch_sec',
       'call_to_arrival_sec', 'dispatch_to_arrival_sec',
       'arrival_to_complete_sec', 'call_to_complete_sec',
       'call_to_impound_sec', 'with_collar', 'with_tether'],
      dtype='object')

In [62]:
#drop the initial date and time columns, location_y
clean_df = clean_df.drop(columns = ['incident_date', 'incident_time', 'dispatched_time', 'arrival_time', 'available_time', 'impound_date', 
                                    'impound_time', 'location_y', 'collar'])

In [63]:
pd.set_option('display.max_columns', None)
clean_df.head()

,request_type,location_x,municipality_,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
0,CRUELTY CASE,3366 SYCAMORE,BATON ROUGE,70805,LOOSE,DOG,PIT BULL,F,LARGE,FAIR,FRIENDLY,TRANS CAA,2016-12-27 15:30:00,2016-12-27 16:06:00,2016-12-27 16:17:00,2016-12-27 16:38:00,2016-12-27 16:20:00,30.494640,-91.156160,2160.0,2820.0,660.0,1260.0,4080.0,3000.0,True,False
1,TRAPPED ANIMAL,2582 WOODLAND RIDGE,BATON ROUGE,70816,TRAPPED ANIMAL,OTHER,POSSUM,U,LARGE,FAIR,NORMAL,RETURN TO WILD,2016-12-28 10:06:00,2016-12-28 10:07:00,2016-12-28 10:15:00,2016-12-28 10:23:00,2016-12-28 10:21:00,30.431417,-91.141070,60.0,540.0,480.0,480.0,1020.0,900.0,False,False
2,STRAY,4463 MAIN,ZACHARY,70791,STRAY AN SECURED,DOG,PIT BULL,M,LARGE,EMACIATED,FRIENDLY,TRANS CAA,2016-12-28 09:03:00,2016-12-28 09:03:00,2016-12-28 09:27:00,2016-12-28 09:45:00,2016-12-28 09:41:00,30.648540,-91.156770,0.0,1440.0,1440.0,1080.0,2520.0,2280.0,True,False
3,STRAY,299 VAN BUREN,BATON ROUGE,70714,STRAY,DOG,YORKSHIRE TERRIER,F,SMALL,EMACIATED,NORMAL,TRANS CAA,2016-12-28 15:05:00,2016-12-28 15:45:00,2016-12-28 16:07:00,2016-12-28 16:25:00,2016-12-28 16:20:00,30.429104,-91.185036,2400.0,3720.0,1320.0,1080.0,4800.0,4500.0,False,False
4,OWNER SIGNED RELEASE,6621 MARIONETTE,BATON ROUGE,70811,OWNER SIGNED RELEASE,DOG,DOBERMAN PINSCHER,F,EXTRA LARGE,GOOD,FRIENDLY,TRANS CAA,2016-12-28 10:03:00,2016-12-28 10:10:00,2016-12-28 10:35:00,2016-12-28 10:52:00,2016-12-28 10:41:00,30.529593,-91.125534,420.0,1920.0,1500.0,1020.0,2940.0,2280.0,True,False


In [64]:
clean_df.disposition.unique()

array(['TRANS CAA', 'RETURN TO WILD', 'UNKNOWN', 'RETURN TO OWNER',
       'SPECIAL CASE/OTHER', 'DEAD ON ARRIVAL', 'EUTHANIZED', nan],
      dtype=object)

In [65]:
clean_df[clean_df.disposition.isna()]

,request_type,location_x,municipality_,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
18864,UNKNOWN,300 S. 19TH,BATON ROUGE,0,INJURED,OTHER,UNKNOWN,U,MEDIUM,FAIR,NORMAL,NaN,2014-11-10 08:09:00,2014-11-10 08:19:00,2014-11-10 08:35:00,2014-11-10 08:41:00,2014-11-10 08:35:00,30.445700,-91.169150,600.0,1560.0,960.0,360.0,1920.0,1560.0,False,False
29201,UNKNOWN,16925 GEORGEE ONEAL,BATON ROUGE,0,SICK ANIMAL,OTHER,RACCOON,U,SMALL,FAIR,NORMAL,NaN,2013-05-09 06:12:00,2013-05-09 08:17:00,2013-05-09 09:09:00,2013-05-09 09:16:00,2013-05-09 09:16:00,30.410471,-90.999870,7500.0,10620.0,3120.0,420.0,11040.0,11040.0,False,False
37187,UNKNOWN,5875 MAIN ST.,BATON ROUGE,0,INJURED,OTHER,UNKNOWN,U,MEDIUM,FAIR,NORMAL,NaN,2014-07-14 13:27:00,2014-07-14 14:11:00,2014-07-14 14:28:00,2014-07-14 14:29:00,2014-07-14 14:30:00,30.452898,-91.150408,2640.0,3660.0,1020.0,60.0,3720.0,3780.0,False,False
39775,UNKNOWN,191 LSU PL,BATON ROUGE,70808,VET PICKUP,DOG,BEAGLE,F,MEDIUM,EMACIATED,NORMAL,NaN,2015-09-28 10:10:00,2015-09-28 10:10:00,2015-09-28 10:23:00,2015-09-28 10:41:00,2015-09-28 10:35:00,30.403093,-91.170165,0.0,780.0,780.0,1080.0,1860.0,1500.0,False,False
42890,UNKNOWN,246 W. WOODSTONE,BATON ROUGE,0,WILD LIVE,OTHER,UNKNOWN,U,SMALL,FAIR,NORMAL,NaN,2013-05-09 13:20:00,2013-05-09 13:33:00,2013-05-09 13:49:00,2013-05-09 13:54:00,2013-05-09 13:51:00,30.387432,-91.148570,780.0,1740.0,960.0,300.0,2040.0,1860.0,False,False
46587,UNKNOWN,22172 SUTTER RD.,BATON ROUGE,0,WILD LIVE,OTHER,UNKNOWN,U,SMALL,FAIR,UNKNOWN,NaN,2013-05-09 14:17:00,2013-05-09 14:59:00,2013-05-09 15:20:00,2013-05-09 15:31:00,2013-05-09 15:27:00,30.451468,-91.187147,2520.0,3780.0,1260.0,660.0,4440.0,4200.0,False,False


In [66]:
#set null dispositions to unknown
clean_df['disposition'] = clean_df['disposition'].fillna(value='UNKNOWN')

In [67]:
clean_df.disposition.unique()

array(['TRANS CAA', 'RETURN TO WILD', 'UNKNOWN', 'RETURN TO OWNER',
       'SPECIAL CASE/OTHER', 'DEAD ON ARRIVAL', 'EUTHANIZED'],
      dtype=object)

In [68]:
clean_df.head()

,request_type,location_x,municipality_,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
0,CRUELTY CASE,3366 SYCAMORE,BATON ROUGE,70805,LOOSE,DOG,PIT BULL,F,LARGE,FAIR,FRIENDLY,TRANS CAA,2016-12-27 15:30:00,2016-12-27 16:06:00,2016-12-27 16:17:00,2016-12-27 16:38:00,2016-12-27 16:20:00,30.494640,-91.156160,2160.0,2820.0,660.0,1260.0,4080.0,3000.0,True,False
1,TRAPPED ANIMAL,2582 WOODLAND RIDGE,BATON ROUGE,70816,TRAPPED ANIMAL,OTHER,POSSUM,U,LARGE,FAIR,NORMAL,RETURN TO WILD,2016-12-28 10:06:00,2016-12-28 10:07:00,2016-12-28 10:15:00,2016-12-28 10:23:00,2016-12-28 10:21:00,30.431417,-91.141070,60.0,540.0,480.0,480.0,1020.0,900.0,False,False
2,STRAY,4463 MAIN,ZACHARY,70791,STRAY AN SECURED,DOG,PIT BULL,M,LARGE,EMACIATED,FRIENDLY,TRANS CAA,2016-12-28 09:03:00,2016-12-28 09:03:00,2016-12-28 09:27:00,2016-12-28 09:45:00,2016-12-28 09:41:00,30.648540,-91.156770,0.0,1440.0,1440.0,1080.0,2520.0,2280.0,True,False
3,STRAY,299 VAN BUREN,BATON ROUGE,70714,STRAY,DOG,YORKSHIRE TERRIER,F,SMALL,EMACIATED,NORMAL,TRANS CAA,2016-12-28 15:05:00,2016-12-28 15:45:00,2016-12-28 16:07:00,2016-12-28 16:25:00,2016-12-28 16:20:00,30.429104,-91.185036,2400.0,3720.0,1320.0,1080.0,4800.0,4500.0,False,False
4,OWNER SIGNED RELEASE,6621 MARIONETTE,BATON ROUGE,70811,OWNER SIGNED RELEASE,DOG,DOBERMAN PINSCHER,F,EXTRA LARGE,GOOD,FRIENDLY,TRANS CAA,2016-12-28 10:03:00,2016-12-28 10:10:00,2016-12-28 10:35:00,2016-12-28 10:52:00,2016-12-28 10:41:00,30.529593,-91.125534,420.0,1920.0,1500.0,1020.0,2940.0,2280.0,True,False


### Final Stats

In [69]:
clean_df.shape

(43388, 27)

In [70]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43388 entries, 0 to 55888
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   request_type             43388 non-null  object        
 1   location_x               43334 non-null  object        
 2   municipality_            43387 non-null  object        
 3   zip_code                 43388 non-null  int64         
 4   service_code             43388 non-null  object        
 5   species                  43388 non-null  object        
 6   breed                    43388 non-null  object        
 7   sex                      34469 non-null  object        
 8   size                     43388 non-null  object        
 9   condition                43388 non-null  object        
 10  temperament              43388 non-null  object        
 11  disposition              43388 non-null  object        
 12  incident_datetime        43388 n

In [71]:
#still some null values
#location_x - rename and drop null observations; this is key to our analysis
#municpality_ - rename and drop null
#sex - view and change null to unknown
#impound_datetime - goingt o leave these because i'm not sure if they are relevant to analysis; 
    #they may end up getting excluded anyway

In [72]:
#renaming
clean_df = clean_df.rename(columns={'location_x':'location'})
clean_df = clean_df.rename(columns={'municipality_':'municipality'})

In [73]:
#drop na
clean_df = clean_df.dropna(axis=0, subset=['location', 'municipality'])
clean_df.shape

(43333, 27)

In [74]:
clean_df.sex.unique()

array(['F', 'U', 'M', nan], dtype=object)

In [75]:
clean_df['sex'] = clean_df['sex'].fillna(value='U')

In [76]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 43333 entries, 0 to 55888
Data columns (total 27 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   request_type             43333 non-null  object        
 1   location                 43333 non-null  object        
 2   municipality             43333 non-null  object        
 3   zip_code                 43333 non-null  int64         
 4   service_code             43333 non-null  object        
 5   species                  43333 non-null  object        
 6   breed                    43333 non-null  object        
 7   sex                      43333 non-null  object        
 8   size                     43333 non-null  object        
 9   condition                43333 non-null  object        
 10  temperament              43333 non-null  object        
 11  disposition              43333 non-null  object        
 12  incident_datetime        43333 n

In [77]:
clean_df.describe().T

,count,mean,std,min,25%,50%,75%,max
zip_code,43333.0,39283.380588,3.518748e+04,0.000000e+00,0.000000,70791.000000,70808.000000,7.801400e+04
lat,43333.0,30.473642,7.382459e-02,3.006421e+01,30.425697,30.463930,30.509064,3.078200e+01
long,43333.0,-91.111057,7.189420e-02,-9.196859e+01,-91.159560,-91.126919,-91.065575,-8.718023e+01
call_to_dispatch_sec,43333.0,1678.021369,8.115798e+03,-6.090000e+04,120.000000,1140.000000,3540.000000,4.986000e+04
call_to_arrival_sec,43333.0,2208.569912,9.444626e+03,-6.402000e+04,1140.000000,2280.000000,4560.000000,4.638000e+04
dispatch_to_arrival_sec,43333.0,530.548543,5.608715e+03,-7.998000e+04,480.000000,840.000000,1320.000000,7.872000e+04
arrival_to_complete_sec,43333.0,1325.958046,5.624899e+03,-7.872000e+04,360.000000,600.000000,1140.000000,6.414000e+04
call_to_complete_sec,43333.0,3534.527958,8.599973e+03,-6.090000e+04,1980.000000,3360.000000,5640.000000,4.002000e+04
call_to_impound_sec,42729.0,-17380.509724,2.483306e+06,-3.155311e+08,1620.000000,2940.000000,5400.000000,2.209291e+08


In [78]:
clean_df.describe(include=np.object).T

,count,unique,top,freq
request_type,43333,31,UNKNOWN,24955
location,43333,21992,2680 PROGRESS,182
municipality,43333,6,BATON ROUGE,38245
service_code,43333,25,TRAPPED ANIMAL,15061
species,43333,3,DOG,21293
breed,43333,128,DSH,9709
sex,43333,3,U,21697
size,43333,5,MEDIUM,19253
condition,43333,6,FAIR,27952
temperament,43333,6,NORMAL,20644


In [79]:
#run through pandas profiling again.
profile = ProfileReport(clean_df, title="BRACRC Data Profiling Report")
profile.to_widgets()

Summarize dataset:   0%|          | 0/41 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

In [80]:
clean_df[clean_df['call_to_dispatch_sec'] < 0]

,request_type,location,municipality,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
6,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DMH,U,LARGE,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01,2016-06-01 00:00:00,2016-06-01,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
7,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01,2016-06-01 00:00:00,2016-06-01,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
8,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01,2016-06-01 00:00:00,2016-06-01,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
9,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01,2016-06-01 00:00:00,2016-06-01,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
10,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,LARGE,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01,2016-06-01 00:00:00,2016-06-01,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55578,STRAY,9313 MARTINIQUE,BATON ROUGE,70810,STRAY,DOG,LABRADOR RETRIEVER,M,EXTRA LARGE,EMACIATED,NERVOUS,TRANS CAA,2017-06-06 13:24:00,2017-06-06,2017-06-06 00:00:00,2017-06-06,2017-06-06 17:50:00,30.345766,-91.127200,-48240.0,-48240.0,0.0,0.0,-48240.0,15960.0,True,False
55634,STRAY,3830 POPE,ZACHARY,70791,STRAY,DOG,PIT BULL,F,MEDIUM,FAIR,NORMAL,TRANS CAA,2018-08-14 16:54:00,2018-08-14,2018-08-14 00:00:00,2018-08-14,2018-08-14 19:17:00,30.659924,-91.162830,-60840.0,-60840.0,0.0,0.0,-60840.0,8580.0,False,False
55670,STRAY,6575 LAKE MARY,ZACHARY,70791,VET PICKUP,DOG,TERRIER,F,SMALL,FAIR,FRIENDLY,TRANS CAA,2017-05-26 13:04:00,2017-05-26,2017-05-26 00:00:00,2017-05-26,2017-05-26 13:20:00,30.648486,-91.189172,-47040.0,-47040.0,0.0,0.0,-47040.0,960.0,False,False
55743,STRAY,2488 BARBER,BATON ROUGE,70808,STRAY,DOG,BORDER COLLIE,M,UNKNOWN,UNKNOWN,FRIENDLY,TRANS CAA,2016-06-13 12:55:00,2016-06-13,2016-06-13 17:43:00,2016-06-13,2016-06-10 17:43:00,30.425722,-91.139150,-46500.0,17280.0,63780.0,-63780.0,-46500.0,-241920.0,False,False


In [81]:
clean_df[clean_df['call_to_arrival_sec'] < 0]

,request_type,location,municipality,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
6,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DMH,U,LARGE,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01 00:00:00,2016-06-01,2016-06-01 00:00:00,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
7,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01 00:00:00,2016-06-01,2016-06-01 00:00:00,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
8,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01 00:00:00,2016-06-01,2016-06-01 00:00:00,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
9,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01 00:00:00,2016-06-01,2016-06-01 00:00:00,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
10,UNKNOWN,2680 PROGRESS,BATON ROUGE,70807,OWNER SIGNED RELEASE,CAT,DSH,U,LARGE,FAIR,NORMAL,TRANS CAA,2016-06-01 13:59:00,2016-06-01 00:00:00,2016-06-01,2016-06-01 00:00:00,2016-06-26 08:45:00,30.537788,-91.164880,-50340.0,-50340.0,0.0,0.0,-50340.0,2141160.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55755,STRAY,6587 MARIONETTE,PARISH,70811,LOOSE,DOG,SHIH TZU,M,SMALL,FAIR,FRIENDLY,TRANS CAA,2018-04-22 08:08:00,2018-04-22 08:46:00,2018-04-22,2018-04-22 08:46:00,2018-04-27 08:25:00,30.529750,-91.125790,2280.0,-29280.0,-31560.0,31560.0,2280.0,433020.0,True,False
55772,STRAY,3124 JUBAN,BATON ROUGE,70802,STRAY,DOG,PIT BULL,M,MEDIUM,FAIR,NORMAL,TRANS CAA,2018-11-09 13:02:00,2018-11-09 13:29:00,2018-11-09,2018-11-09 13:55:00,2018-11-09 14:21:00,30.477943,-91.121660,1620.0,-46920.0,-48540.0,50100.0,3180.0,4740.0,True,False
55789,STRAY,1555 79TH,BATON ROUGE,70807,STRAY,DOG,LABRADOR RETRIEVER,F,MEDIUM,FAIR,FRIENDLY,TRANS CAA,2017-11-06 16:47:00,2017-11-06 16:47:00,2017-11-06,2017-11-06 16:49:00,2017-11-06 17:00:00,30.519500,-91.176440,0.0,-60420.0,-60420.0,60540.0,120.0,780.0,False,False
55855,STRAY,4650 OSBORNE,BATON ROUGE,70805,STRAY AN SECURED,DOG,PIT BULL,F,LARGE,FAIR,FRIENDLY,TRANS CAA,2018-09-13 10:19:00,2018-09-13 16:31:00,2018-09-13,2018-09-13 16:56:00,2018-09-13 09:19:00,30.473137,-91.143036,22320.0,-37140.0,-59460.0,60960.0,23820.0,-3600.0,True,False


In [82]:
clean_df[clean_df['dispatch_to_arrival_sec'] < 0]

,request_type,location,municipality,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
376,NON-DOMESTIC ANIMALS,3838 THOMAS #9,BATON ROUGE,70811,WILD LIVE,OTHER,POSSUM,U,MEDIUM,FAIR,NORMAL,RETURN TO WILD,2016-05-10 16:50:00,2016-05-10 17:38:00,2016-05-10,2016-05-10 00:00:00,2016-05-10 17:38:00,30.558903,-91.153020,2880.0,-60600.0,-63480.0,0.0,-60600.0,2880.0,False,False
409,STRAY,4980 LOWER ZACHARY,ZACHARY,70791,STRAY AN SECURED,CAT,DSH,U,MEDIUM,FAIR,NERVOUS,TRANS CAA,2017-01-20 10:46:00,2017-01-20 13:27:00,2017-01-20,2017-01-20 13:44:00,2017-01-20 13:41:00,30.641096,-91.152220,9660.0,-38760.0,-48420.0,49440.0,10680.0,10500.0,False,False
428,INJURED ANIMAL,1101 REIGER,BATON ROUGE,0,INJURED,OTHER,POSSUM,U,MEDIUM,FAIR,NORMAL,RETURN TO WILD,2016-05-11 08:22:00,2016-05-11 22:13:00,2016-05-11,2016-05-11 00:00:00,2016-05-10 22:13:00,30.377102,-91.057108,49860.0,-30120.0,-79980.0,0.0,-30120.0,-36540.0,False,False
872,INJURED ANIMAL,16438 VERMILLION,BATON ROUGE,70814,ANIMAL RESCUE,DOG,PIT BULL,M,LARGE,FAIR,FRIENDLY,TRANS CAA,2017-02-07 16:33:00,2017-02-07 16:36:00,2017-02-07,2017-02-07 16:55:00,2017-02-07 17:15:00,30.476257,-91.006290,180.0,-59580.0,-59760.0,60900.0,1320.0,2520.0,True,False
1338,STRAY,276 FOUNTAINDLEAU,BATON ROUGE,70819,STRAY AN SECURED,DOG,TERRIER,M,SMALL,GOOD,FRIENDLY,TRANS CAA,2017-03-13 13:05:00,2017-03-13 13:35:00,2017-03-13,2017-03-13 14:08:00,2017-03-13 13:52:00,30.462643,-91.019770,1800.0,-47100.0,-48900.0,50880.0,3780.0,2820.0,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55741,PITBULL,4747 RITTERMAN,BATON ROUGE,70805,STRAY,DOG,PIT BULL,M,LARGE,FAIR,NORMAL,RETURN TO OWNER,2018-04-02 16:07:00,2018-04-02 16:30:00,2018-04-02,2018-04-02 16:55:00,2018-04-02 17:00:00,30.474820,-91.142480,1380.0,-58020.0,-59400.0,60900.0,2880.0,3180.0,True,False
55755,STRAY,6587 MARIONETTE,PARISH,70811,LOOSE,DOG,SHIH TZU,M,SMALL,FAIR,FRIENDLY,TRANS CAA,2018-04-22 08:08:00,2018-04-22 08:46:00,2018-04-22,2018-04-22 08:46:00,2018-04-27 08:25:00,30.529750,-91.125790,2280.0,-29280.0,-31560.0,31560.0,2280.0,433020.0,True,False
55772,STRAY,3124 JUBAN,BATON ROUGE,70802,STRAY,DOG,PIT BULL,M,MEDIUM,FAIR,NORMAL,TRANS CAA,2018-11-09 13:02:00,2018-11-09 13:29:00,2018-11-09,2018-11-09 13:55:00,2018-11-09 14:21:00,30.477943,-91.121660,1620.0,-46920.0,-48540.0,50100.0,3180.0,4740.0,True,False
55789,STRAY,1555 79TH,BATON ROUGE,70807,STRAY,DOG,LABRADOR RETRIEVER,F,MEDIUM,FAIR,FRIENDLY,TRANS CAA,2017-11-06 16:47:00,2017-11-06 16:47:00,2017-11-06,2017-11-06 16:49:00,2017-11-06 17:00:00,30.519500,-91.176440,0.0,-60420.0,-60420.0,60540.0,120.0,780.0,False,False


In [83]:
#delete negative time differences
drop_indexes = clean_df.index[(clean_df['call_to_dispatch_sec'] < 0) | (clean_df['call_to_arrival_sec'] < 0 ) | 
                              (clean_df['dispatch_to_arrival_sec'] < 0) | (clean_df['arrival_to_complete_sec'] < 0) | 
                              (clean_df['call_to_complete_sec'] < 0) | (clean_df['call_to_impound_sec'] < 0)]

clean_df = clean_df.drop(drop_indexes)


In [84]:
clean_df = clean_df.reset_index(drop=True)
clean_df.head()

,request_type,location,municipality,zip_code,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
0,CRUELTY CASE,3366 SYCAMORE,BATON ROUGE,70805,LOOSE,DOG,PIT BULL,F,LARGE,FAIR,FRIENDLY,TRANS CAA,2016-12-27 15:30:00,2016-12-27 16:06:00,2016-12-27 16:17:00,2016-12-27 16:38:00,2016-12-27 16:20:00,30.494640,-91.156160,2160.0,2820.0,660.0,1260.0,4080.0,3000.0,True,False
1,TRAPPED ANIMAL,2582 WOODLAND RIDGE,BATON ROUGE,70816,TRAPPED ANIMAL,OTHER,POSSUM,U,LARGE,FAIR,NORMAL,RETURN TO WILD,2016-12-28 10:06:00,2016-12-28 10:07:00,2016-12-28 10:15:00,2016-12-28 10:23:00,2016-12-28 10:21:00,30.431417,-91.141070,60.0,540.0,480.0,480.0,1020.0,900.0,False,False
2,STRAY,4463 MAIN,ZACHARY,70791,STRAY AN SECURED,DOG,PIT BULL,M,LARGE,EMACIATED,FRIENDLY,TRANS CAA,2016-12-28 09:03:00,2016-12-28 09:03:00,2016-12-28 09:27:00,2016-12-28 09:45:00,2016-12-28 09:41:00,30.648540,-91.156770,0.0,1440.0,1440.0,1080.0,2520.0,2280.0,True,False
3,STRAY,299 VAN BUREN,BATON ROUGE,70714,STRAY,DOG,YORKSHIRE TERRIER,F,SMALL,EMACIATED,NORMAL,TRANS CAA,2016-12-28 15:05:00,2016-12-28 15:45:00,2016-12-28 16:07:00,2016-12-28 16:25:00,2016-12-28 16:20:00,30.429104,-91.185036,2400.0,3720.0,1320.0,1080.0,4800.0,4500.0,False,False
4,OWNER SIGNED RELEASE,6621 MARIONETTE,BATON ROUGE,70811,OWNER SIGNED RELEASE,DOG,DOBERMAN PINSCHER,F,EXTRA LARGE,GOOD,FRIENDLY,TRANS CAA,2016-12-28 10:03:00,2016-12-28 10:10:00,2016-12-28 10:35:00,2016-12-28 10:52:00,2016-12-28 10:41:00,30.529593,-91.125534,420.0,1920.0,1500.0,1020.0,2940.0,2280.0,True,False


In [85]:
clean_df = clean_df.drop(columns='zip_code')

In [86]:
clean_df.head()

,request_type,location,municipality,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
0,CRUELTY CASE,3366 SYCAMORE,BATON ROUGE,LOOSE,DOG,PIT BULL,F,LARGE,FAIR,FRIENDLY,TRANS CAA,2016-12-27 15:30:00,2016-12-27 16:06:00,2016-12-27 16:17:00,2016-12-27 16:38:00,2016-12-27 16:20:00,30.494640,-91.156160,2160.0,2820.0,660.0,1260.0,4080.0,3000.0,True,False
1,TRAPPED ANIMAL,2582 WOODLAND RIDGE,BATON ROUGE,TRAPPED ANIMAL,OTHER,POSSUM,U,LARGE,FAIR,NORMAL,RETURN TO WILD,2016-12-28 10:06:00,2016-12-28 10:07:00,2016-12-28 10:15:00,2016-12-28 10:23:00,2016-12-28 10:21:00,30.431417,-91.141070,60.0,540.0,480.0,480.0,1020.0,900.0,False,False
2,STRAY,4463 MAIN,ZACHARY,STRAY AN SECURED,DOG,PIT BULL,M,LARGE,EMACIATED,FRIENDLY,TRANS CAA,2016-12-28 09:03:00,2016-12-28 09:03:00,2016-12-28 09:27:00,2016-12-28 09:45:00,2016-12-28 09:41:00,30.648540,-91.156770,0.0,1440.0,1440.0,1080.0,2520.0,2280.0,True,False
3,STRAY,299 VAN BUREN,BATON ROUGE,STRAY,DOG,YORKSHIRE TERRIER,F,SMALL,EMACIATED,NORMAL,TRANS CAA,2016-12-28 15:05:00,2016-12-28 15:45:00,2016-12-28 16:07:00,2016-12-28 16:25:00,2016-12-28 16:20:00,30.429104,-91.185036,2400.0,3720.0,1320.0,1080.0,4800.0,4500.0,False,False
4,OWNER SIGNED RELEASE,6621 MARIONETTE,BATON ROUGE,OWNER SIGNED RELEASE,DOG,DOBERMAN PINSCHER,F,EXTRA LARGE,GOOD,FRIENDLY,TRANS CAA,2016-12-28 10:03:00,2016-12-28 10:10:00,2016-12-28 10:35:00,2016-12-28 10:52:00,2016-12-28 10:41:00,30.529593,-91.125534,420.0,1920.0,1500.0,1020.0,2940.0,2280.0,True,False


In [87]:
#service code - combine the bite cases, stray, wildlife/livestock
clean_df.service_code.unique()

array(['LOOSE', 'TRAPPED ANIMAL', 'STRAY AN SECURED', 'STRAY',
       'OWNER SIGNED RELEASE', 'INJURED', 'BITE CASE LOOSE',
       'LOOSE LIVESTOCK', 'SICK ANIMAL', 'ANIMAL RESCUE', 'CRUELTY',
       'DANGEROUS ANIMAL SIT', 'VET PICKUP', 'UNKNOWN',
       'UNIT STANDING BY', 'WILD LIVE', 'SNAKE', 'SPECIAL CASE',
       'BITE CASE SECURE', 'ABANDONED ANIMAL', 'SCHOOL CALL',
       'DOG FIGHTING', 'BITE CASE', 'OWNED', 'HIT BY CAR'], dtype=object)

In [88]:
clean_df['service_code'] = clean_df['service_code'].replace('STRAY AN SECURED', 'STRAY')
clean_df['service_code'] = clean_df['service_code'].replace(['INJURED', 'SICK ANIMAL', 'HIT BY CAR'], 'INJURED/SICK')
clean_df['service_code'] = clean_df['service_code'].replace(['BITE CASE LOOSE', 'BITE CASE SECURE'], 'BITE CASE')
clean_df['service_code'] = clean_df['service_code'].replace(['LOOSE LIVESTOCK', 'WILD LIVE', 'SNAKE'], 'NON-DOMESTIC ANIMALS')

In [89]:
clean_df.service_code.unique()

array(['LOOSE', 'TRAPPED ANIMAL', 'STRAY', 'OWNER SIGNED RELEASE',
       'INJURED/SICK', 'BITE CASE', 'NON-DOMESTIC ANIMALS',
       'ANIMAL RESCUE', 'CRUELTY', 'DANGEROUS ANIMAL SIT', 'VET PICKUP',
       'UNKNOWN', 'UNIT STANDING BY', 'SPECIAL CASE', 'ABANDONED ANIMAL',
       'SCHOOL CALL', 'DOG FIGHTING', 'OWNED'], dtype=object)

In [90]:
#save before removing outliers so you can get an image with outliers.
clean_df.to_csv('../Data/dataset_before_outliers.csv', index=False)

### Outlier latitude and longitudes
Some latitudes and longitudes are outside of east baton rouge parish.  

lat boundaries - 30.31 to 30.71
long boundaries - -90.29 to -90.86

In [91]:
outliers = clean_df[(clean_df['lat'] < 30.31) | (clean_df['lat'] > 30.72) | (clean_df['long'] < -91.29) | (clean_df['long'] > -90.86)]
outliers.head()

,request_type,location,municipality,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether
1884,STRAY,21540 PORT HICKEY,PARISH,STRAY,DOG,PIT BULL,M,LARGE,FAIR,FRIENDLY,TRANS CAA,2017-06-08 08:10:00,2017-06-08 08:21:00,2017-06-08 08:45:00,2017-06-08 08:57:00,2017-06-08 08:50:00,30.663937,-91.290050,660.0,2100.0,1440.0,720.0,2820.0,2400.0,False,False
7526,STRAY,21540 PORT HICKEY,ZACHARY,STRAY,DOG,PIT BULL,M,MEDIUM,FAIR,NERVOUS,TRANS CAA,2018-11-19 09:13:00,2018-11-19 09:15:00,2018-11-19 09:41:00,2018-11-19 10:00:00,2018-11-19 09:53:00,30.663937,-91.290050,120.0,1680.0,1560.0,1140.0,2820.0,2400.0,False,False
8029,TRAPPED ANIMAL,1356 HWY 19,ZACHARY,TRAPPED ANIMAL,CAT,DSH,U,SMALL,FAIR,NORMAL,TRANS CAA,2019-01-23 12:15:00,2019-01-23 13:32:00,2019-01-23 13:53:00,2019-01-23 14:02:00,2019-01-23 13:56:00,30.734037,-91.139827,4620.0,5880.0,1260.0,540.0,6420.0,6060.0,False,False
10866,UNKNOWN,21510 PORT HICKEY,BATON ROUGE,TRAPPED ANIMAL,CAT,DSH,U,MEDIUM,FAIR,NORMAL,TRANS CAA,2012-09-10 10:14:00,2012-09-10 10:32:00,2012-09-10 11:23:00,2012-09-10 11:29:00,2012-09-10 11:27:00,30.663283,-91.290210,1080.0,4140.0,3060.0,360.0,4500.0,4380.0,False,False
11264,UNKNOWN,18866 FERNCREST,BATON ROUGE,TRAPPED ANIMAL,OTHER,POSSUM,U,LARGE,FAIR,NORMAL,RETURN TO WILD,2015-09-24 11:24:00,2015-09-24 13:27:00,2015-09-24 13:54:00,2015-09-24 14:02:00,2015-09-24 14:00:00,30.071023,-89.948203,7380.0,9000.0,1620.0,480.0,9480.0,9360.0,False,False


In [92]:
outliers.shape

(23, 26)

In [93]:
outliers.index

Int64Index([ 1884,  7526,  8029, 10866, 11264, 13191, 14194, 15368, 18405,
            18804, 18805, 19037, 19140, 19859, 23071, 23817, 26978, 27061,
            31307, 32092, 32986, 34863, 35906],
           dtype='int64')

In [94]:
clean_df = clean_df.drop(outliers.index)

In [95]:
clean_df.shape

(38288, 26)

In [96]:
#save data set and start visualizing
clean_df.to_csv('../Data/Final_dataset.csv', index=False)

In [97]:
clean_df[clean_df.species.isna()]

,request_type,location,municipality,service_code,species,breed,sex,size,condition,temperament,disposition,incident_datetime,dispatched_datetime,arrival_datetime,available_datetime,impound_datetime,lat,long,call_to_dispatch_sec,call_to_arrival_sec,dispatch_to_arrival_sec,arrival_to_complete_sec,call_to_complete_sec,call_to_impound_sec,with_collar,with_tether


In [98]:
profile.to_file('../Data/BRACRC Profile Report.html')

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]